In [1]:
import os
import torch
from torch import nn
import numpy as np
import random
from dataset import ModelnetDataset
from pathlib import Path
from torch.utils.data import DataLoader


from util import setup_seed, train, test, train_HHN, test_HHN, train_one4one, count_parameters, save_checkpoint
from model import One4All, HyperNetwork
from render import PointcloudRender

from torch.utils.tensorboard import SummaryWriter

In [2]:
batch_size = 256 #  
epochs = 6000 #
best_acc1 = 0
start_epoch = 0
dimension = 6
lr=0.006
# 
from datetime import datetime
import pytz
today = datetime.now(pytz.timezone('Europe/Vienna')).strftime('%m-%d')

In [3]:
setup_seed(666)
device_for_dataset = "cuda:8" if torch.cuda.is_available() else "cpu"
writer = SummaryWriter(f'tf-logs/HHN/bs{batch_size}_lr{lr}_{int(epochs/1000)}k_D_{dimension}_hyperwide72')


In [4]:
path = Path("./data")
train_ds = ModelnetDataset(path, folder="train", device=device_for_dataset) # Just put dataset in mem or GPU
test_ds = ModelnetDataset(path, folder="test", device=device_for_dataset) # Just put dataset in mem or GPU
classes = {i: cat for cat, i in train_ds.classes.items()}

print('Train dataset size: ', len(train_ds))
print('Test dataset size: ', len(test_ds))
print('Number of classes: ', len(train_ds.classes))
print('Class: ', classes[train_ds[11]['category']])

Preocessing train data in_mem = True and infor: Default infor


Prepare train dataset categories: 100%|██████████| 10/10 [08:10<00:00, 49.06s/it]


Preocessing test data in_mem = True and infor: Default infor


Prepare test dataset categories: 100%|██████████| 10/10 [01:30<00:00,  9.07s/it]

Train dataset size:  3991
Test dataset size:  908
Number of classes:  10
Class:  bathtub


In [5]:
from collections import Counter
print(dict(Counter(sample['category'] for sample in train_ds.files)))

print(dict(Counter(sample['category'] for sample in test_ds.files)))



{'bathtub': 106, 'bed': 515, 'chair': 889, 'desk': 200, 'dresser': 200, 'monitor': 465, 'night_stand': 200, 'sofa': 680, 'table': 392, 'toilet': 344}
{'bathtub': 50, 'bed': 100, 'chair': 100, 'desk': 86, 'dresser': 86, 'monitor': 100, 'night_stand': 86, 'sofa': 100, 'table': 100, 'toilet': 100}


In [6]:
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=True)


In [7]:
# day_path = f'checkpoint/HHN/{today}'
# print(f'sdsdsdsd{day_path}')

In [8]:
day_path = f'checkpoint/HHN/{today}'

model_checkpoints_path = f'{day_path}/bs{batch_size}_lr{lr}_{int(epochs/1000)}k_D_{dimension}'

if not os.path.exists(day_path):
    print(f'Creating {day_path} ...')
    os.makedirs(day_path)
if not os.path.exists(model_checkpoints_path):
    print(f'Creating {model_checkpoints_path} ...')
    os.makedirs(model_checkpoints_path)
print(f'Models will be saved in {model_checkpoints_path}')
    

Models will be saved in checkpoint/HHN/12-30/bs256_lr0.006_6k_D_6


In [9]:
best_filename = f'{model_checkpoints_path}/Best-bs{batch_size}_lr{lr}_{int(epochs/1000)}k_D_{dimension}-HHNcheckpoint.pth.tar'

gpu_computation = "cuda:8" if torch.cuda.is_available() else "cpu"

model = HyperNetwork(dimensions=dimension, hidden_units=72)
model = model.to(gpu_computation)
print(f'Num of model parameter: {count_parameters(model)}')
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = 1e-6)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.004)
# optimizer = torch.optim.SGD(model.parameters(), 0.001,
#                                 momentum=0.9,
#                                 weight_decay=1e-4
#                            )

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=5e-6)
pr = PointcloudRender(              
                 img_size=128,
                 radius=0.0045,
                 points_per_pixel=40,
                 num_points = 4096)

print(model)


Num of model parameter: 832314
HyperNetwork(
  (hyper_stack): Sequential(
    (0): Linear(in_features=6, out_features=72, bias=True)
    (1): ReLU()
    (2): Linear(in_features=72, out_features=6, bias=True)
    (3): Softmax(dim=0)
  )
  (w_conv1_list): ParameterList(
      (0): Parameter containing: [torch.cuda.FloatTensor of size 4x1x5x5 (GPU 8)]
      (1): Parameter containing: [torch.cuda.FloatTensor of size 4x1x5x5 (GPU 8)]
      (2): Parameter containing: [torch.cuda.FloatTensor of size 4x1x5x5 (GPU 8)]
      (3): Parameter containing: [torch.cuda.FloatTensor of size 4x1x5x5 (GPU 8)]
      (4): Parameter containing: [torch.cuda.FloatTensor of size 4x1x5x5 (GPU 8)]
      (5): Parameter containing: [torch.cuda.FloatTensor of size 4x1x5x5 (GPU 8)]
  )
  (bias_conv1_list): ParameterList(
      (0): Parameter containing: [torch.cuda.FloatTensor of size 4 (GPU 8)]
      (1): Parameter containing: [torch.cuda.FloatTensor of size 4 (GPU 8)]
      (2): Parameter containing: [torch.cuda.Fl

In [ ]:
torch.cuda.empty_cache()


angles = torch.randint(-180, 180, (3,))/180*torch.pi
print(f'Test angles: {angles}')

for t in range(epochs):
    print(60*"="+f"\n Epoch: {t + 1} \n"+60*"=")
    # angles = torch.randint(-180, 180, (3,))/180*torch.pi
    
    train_acc, train_loss= train_HHN(train_loader, model, loss_fn, optimizer, t, pr, device=gpu_computation)
    test_acc, test_loss = test_HHN(test_loader, model, loss_fn, pr, device=gpu_computation, angles=angles, verbose=False)
    writer.add_scalar("Loss/train", train_loss, t)
    writer.add_scalar("Accuracy/train", train_acc, t)
    
    writer.add_scalar("Loss/test", test_loss, t)
    writer.add_scalar("Accuracy/test", test_acc, t)
    
    is_best = test_acc > best_acc1
    best_acc1 = max(test_acc, best_acc1)

    if (t+1)%100 == 0:
        save_checkpoint({
        'epoch': t + 1,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer': optimizer.state_dict(),
        }, is_best,
        filename= f'{model_checkpoints_path}/E-{t}-bs{batch_size}_lr{lr}_{int(epochs/1000)}k_D_{dimension}-HHNcheckpoint.pth.tar', #saved_filename, #Todo add GPU and epoch information to the filename
        best_filename = best_filename
        )
    writer.flush()
    lr_scheduler.step()

print("Done!")
writer.close()